# HowTo : Modelling sun and sky light sources

This notebook explain how the natural lighting of a scene by can be achieved. It focuses on the modeling of sources, as , once sources are known, the computation of light interfception is done the same way as before.

## Imports and setup location

In [1]:
%gui qt4
import sky_sources as skys
import light_it as ltfs

Specify a location on earth:

In [2]:
Montpellier ={
'longitude': 3.87,
'latitude': 43.61,
'altitude': 56,
'timezone': 'Europe/Paris'}

create a scene to look at and test lighting

In [3]:
lsys, lstring, lscene = ltfs.run_lsystem('leafy.lpy')

get some meteorological data

In [4]:
meteo_data = skys.montpellier_spring_2013()

## General principles

 - By convention, for processes occuring at earth surface (convention are different in astronomy !), the radiation fluxes are not expressed in terms of radiance of sources (total energetic flux emiited by the source), but in terms of irradiance of a horizontal unit placed at earth surface(total energetic flux passing through a square meter of ground). 
 - The standard measurement of radiation at earth surface is the global horizontal irradiance (ghi, W.m-2.s-1)), that captures radiations coming from the sun and from the sky in the shortwave domain (visible light)
 - The sun contribution is called direct irradiance, whereas sky contribution is called diffuse irradiance
 - Instantaneous direct irradiance can be represented by one light source (the sun)
 - Diffuse irradiance can be represented by several light sources sampling the sky hemisphere. The contribution of each of these sources depends on weather condition (sunny, cloudy, ...) and can be determined by using a sky luminance model.
 - Direct irradiance varies with time, and so does the position of the sun
 - Diffuse irradiance varies with time, but the poistion of the sources can be kept constant. 


## Simulating difuse light conditions

   - When the sky is very cloudy, direct light equals zero, and diffuse light is almost uniform among sky sources as long as cloudy condition last. The absolute value of global horizontal irradiance is varying with time or between two days, but the relative contributions of the different sky sources to ghi does not vary.
   - The positioning and the relative contribution of sky sources to global horizontal irradiance for cloudy conditions can be obtained by the following function that used the 'standard overcast condition' sky luminance model, and make the integration of luminance so that one source capture the portion of sky arround it:

In [5]:
sky = skys.sky_sources(turtle_sectors=16)
skys.show_sources(sky)
elevation, azimuth, irradiance = sky
irradiance

array([0.15986923, 0.04397197, 0.04397197, 0.04397197, 0.04397197,
       0.04397197, 0.11305245, 0.11305245, 0.11305245, 0.11305245,
       0.11305245, 0.01100174, 0.01100174, 0.01100174, 0.01100174,
       0.01100174])

  - turtle sector indicate the number of sources sampling the sky hemisphere (possible values are 6, 16 and 46)

In [6]:
sum(irradiance)

1.0000000000000002

- Lighting the scene with such light is done like before, yielding the distribution of irradiance within the plant

In [7]:
cscene, raw, agg = ltfs.illuminate(lscene, light=sky, scene_unit='cm')
scene, _ = cscene.plot(raw, maxval=1.01)
skys.show_sources(sky,scene=scene, distance=15, radius=0.3)

* Muliplying the results by ghi (instantaneous or averaged over time) yields absolute irradiance values on organs

In [9]:
ghi = 70 #instanteneous, or time averaged global irradiance (W.m-2)
irradiances = ltfs.leaf_irradiance(agg, lstring, leaf_name='Leaf') # normalised irradiances (unitless)
absolute_irradiances = irradiances['Ei'] * ghi # actual irradiances (W.m-2)
absolute_irradiances

4     43.001419
10    43.032468
16    43.013278
22    43.025709
28    43.015932
34    43.026924
40    43.011095
46    43.029272
52    51.017842
58    52.116225
Name: Ei, dtype: float64

## Simulating clear sky conditions

* On clear days, 80% of the radiation is coming from the sun, that is from the south-oriented hemisphere. Moreover, sun irradiance is also varying as a function of time of the day.
* In such case, it is recomended to add sun sources in addition to sky sources. Generaly one position per hour is added 

In [6]:
sky = skys.sky_sources('clear_sky', daydate='2000-06-21', turtle_sectors=16, **Montpellier) 
sun = skys.sun_sources('clear_sky', daydate='2000-06-21', **Montpellier)
sources = skys.add_sources(sun, sky)
#
cscene, raw, agg = ltfs.illuminate(lscene, light=sources, scene_unit='cm', north=180)
scene, _ = cscene.plot(raw)
skys.show_sources(sources,scene=scene, distance=15, radius=0.3, north=180)

- north is the angle (deg, positive clockwise) from X+ to North. It should be pass both to illuminate AND show_source function

## Simulating actual irradiances

In general, meteorological conditions are between overcast and clear sky conditions. The cursor between theses two extremes is a function of the ratio between actual irradiance (measured) and clear sky irradiance. 

In [11]:
observed = skys.actual_irradiance('2013-05-26', meteo_data)
observed

2013-05-26 00:00:00+02:00      0.000000
2013-05-26 01:00:00+02:00      0.000000
2013-05-26 02:00:00+02:00      0.000000
2013-05-26 03:00:00+02:00      0.000000
2013-05-26 04:00:00+02:00      0.000000
2013-05-26 05:00:00+02:00      0.000000
2013-05-26 06:00:00+02:00     31.615385
2013-05-26 07:00:00+02:00    166.370370
2013-05-26 08:00:00+02:00    335.425926
2013-05-26 09:00:00+02:00    505.481481
2013-05-26 10:00:00+02:00    660.689655
2013-05-26 11:00:00+02:00    783.516667
2013-05-26 12:00:00+02:00    865.300000
2013-05-26 13:00:00+02:00    887.883333
2013-05-26 14:00:00+02:00    843.233333
2013-05-26 15:00:00+02:00    861.366667
2013-05-26 16:00:00+02:00    700.766667
2013-05-26 17:00:00+02:00    576.966667
2013-05-26 18:00:00+02:00    415.766667
2013-05-26 19:00:00+02:00    241.550000
2013-05-26 20:00:00+02:00     68.566667
2013-05-26 21:00:00+02:00      1.433333
2013-05-26 22:00:00+02:00      0.000000
2013-05-26 23:00:00+02:00      0.000000
2013-05-27 00:00:00+02:00      0.000000


In [12]:
sun, sky = skys.sun_sky_sources(ghi=observed, dates=observed.index, normalisation=1, **Montpellier) 
sources = skys.add_sources(sun, sky)
#
cscene, raw, agg = ltfs.illuminate(lscene, light=sources, scene_unit='cm', north=180)
scene, _ = cscene.plot(raw)
skys.show_sources(sources,scene=scene, distance=15, radius=0.3, north=180)